In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from scipy.stats import mannwhitneyu

In [2]:
BF_df_CTU = pd.read_feather("BF_df_CTU.feather")
BF_df_OU = pd.read_feather("BF_df_OU.feather")
BF_df_V = pd.read_feather("BF_df_V.feather")
DT_df_CC = pd.read_feather("DT_df_CC.feather")
DT_df_JI = pd.read_feather("DT_df_JI.feather")
IADQ_df = pd.read_feather("IADQ_df.feather")
IESR_df = pd.read_feather("IESR_df.feather")
NAQ_R_df = pd.read_feather("NAQ_R_df.feather")
PCL5_df = pd.read_feather("PCL5_df.feather")
PHQ9_GAD7_df = pd.read_feather("PHQ9_GAD7_df.feather")
PID5_df = pd.read_feather("PID5_df.feather")
PRFQ_df = pd.read_feather("PRFQ_df.feather")
PRMQ_df = pd.read_feather("PRMQ_df.feather")
R_NEO_PI = pd.read_feather("R_NEO_PI.feather")
RAW_DDDT = pd.read_feather("RAW_DDDT.feather")
sPID_5_df = pd.read_feather("sPID-5_df.feather")

In [3]:
dfs = [BF_df_CTU, BF_df_OU, BF_df_V, DT_df_CC, DT_df_JI, IADQ_df, IESR_df, NAQ_R_df, PCL5_df, PHQ9_GAD7_df, PID5_df, PRFQ_df, PRMQ_df, R_NEO_PI, RAW_DDDT, sPID_5_df]

In [4]:
dfs_2 = [BF_df_CTU, BF_df_OU, BF_df_V, DT_df_CC, DT_df_JI, IADQ_df, IESR_df, NAQ_R_df, PCL5_df, PHQ9_GAD7_df, PID5_df, PRFQ_df, PRMQ_df, R_NEO_PI, RAW_DDDT, sPID_5_df]

In [5]:
dfs_names = ['BF_df_CTU', 'BF_df_OU', 'BF_df_V', 'DT_df_CC', 'DF_df_JI', 'IADQ_df', 'IESR_df', 'NAQ_R_df', 'PCL5_df', 'PHQ9_GAD7_df', 'PID5_df', 'PRFQ_df', 'PRMQ_df', 'R_NEO_PI', 'RAQ_DDDT', 'sPID-5_df']

In [ ]:
for i in dfs:
    n, p = i.shape
    print(n, p)

In [6]:
for i in dfs:
    i['CONDITION'] = i['CONDITION'].replace(['H', 'D', 'HONEST', 'DISHONEST', 'FAKE'], ['1', '0', '1', '0', '0'])

### Accuracy of complete datasets

In [7]:
def complete_ds_acc(dfs, dfs_names, data):
    for i in range(len(dfs)):
        
        X = dfs[i].iloc[:,:-1]
        y = dfs[i].iloc[:,-1]
        center_function = lambda x: x - x.mean()
        X = center_function(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # LOGISTIC REGRESSION
        lr = LogisticRegression(C=0.01, random_state=42)
        lr.fit(X_train, y_train)
        y_lr_pred = lr.predict(X_test)
        acc_lr = accuracy_score(y_test, y_lr_pred)

        # RANDOM FOREST
        rf = RandomForestClassifier(random_state=42)
        rf.fit(X_train, y_train)
        y_rf_pred = rf.predict(X_test)
        acc_rf = accuracy_score(y_test, y_rf_pred)

        #SVM
        svm = LinearSVC(loss='hinge',random_state=42, max_iter = 100000)
        svm.fit(X_train, y_train)
        y_pred_svm = svm.predict(X_test)
        acc_svm = accuracy_score(y_pred_svm,y_test)

        data[dfs_names[i]] = [acc_lr,acc_rf, acc_svm]

        
    return data

In [8]:
dic_acc = {}
results_acc = complete_ds_acc(dfs_2, dfs_names, dic_acc)
acc_df = pd.DataFrame(results_acc)
acc_df.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


,BF_df_CTU,BF_df_OU,BF_df_V,DT_df_CC,DF_df_JI,IADQ_df,IESR_df,NAQ_R_df,PCL5_df,PHQ9_GAD7_df,PID5_df,PRFQ_df,PRMQ_df,R_NEO_PI,RAQ_DDDT,sPID-5_df
0,0.797,0.841,0.740,0.717,0.665,0.844,0.963,0.977,0.835,0.988,0.972,0.931,0.912,0.907,0.747,0.968
1,0.827,0.877,0.747,0.731,0.608,0.807,0.907,0.958,0.843,0.982,0.972,0.926,0.931,0.914,0.743,0.965
2,0.805,0.826,0.733,0.683,0.662,0.844,0.898,0.972,0.851,0.985,0.931,0.936,0.903,0.907,0.753,0.968


### Mann-Whitney U test

In [ ]:
### soglia 0.01 o 0.05 ? 


for i in range(len(dfs)):
    n, p = dfs[i].shape
    print(p-1)
    X_honest = dfs[i][dfs[i]['CONDITION'] == '1'].iloc[:, :-1]
    X_dishonest = dfs[i][dfs[i]['CONDITION'] == '0'].iloc[:, :-1]
    indices = []
    for k in range(0, (p-1)):
        U1, s = mannwhitneyu(X_honest.iloc[:, k], X_dishonest.iloc[:, k])
        if s < 0.01:
            indices.append(k)
    print(len(indices))
    print(' ')

In [ ]:
def new_mwu_data(dfs):
    for i in range(len(dfs)):
        n, p = dfs[i].shape
        X_honest = dfs[i][dfs[i]['CONDITION'] == '1'].iloc[:, :-1]
        X_dishonest = dfs[i][dfs[i]['CONDITION'] == '0'].iloc[:, :-1]
        indices = []
        for k in range(0, (p-1)):
            U1, s = mannwhitneyu(X_honest.iloc[:, k], X_dishonest.iloc[:, k])
            if s < 0.01:
                indices.append(k)
        indices.append(-1)
        dfs[i] = dfs[i].iloc[:, indices]
        print(dfs[i])

In [ ]:
new_mwu_data(dfs)

### PCA plots

In [ ]:
for i in dfs:
    X = i.iloc[:,:-1]
    y = i.iloc[:,-1]
    center_function = lambda x: x - x.mean()
    X = center_function(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    pca = PCA(0.70)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    print(pca.explained_variance_ratio_)
    plt.bar(
    range(1,len(pca.explained_variance_)+1),
    pca.explained_variance_
    )
 
    plt.plot(
    range(1,len(pca.explained_variance_ )+1),
    np.cumsum(pca.explained_variance_),
    c='red',
    label='Cumulative Explained Variance')
 
    plt.legend(loc='upper left')
    plt.xlabel('Number of components')
    plt.ylabel('Explained variance (eignenvalues)')
    plt.title('Scree plot')
 
    plt.show()

### Accuracy of datasets after PCA

In [ ]:
def pca_ds_acc(dfs,dfs_names,data):
    for i in range(len(dfs)):
        
        X = dfs[i].iloc[:,:-1]
        y = dfs[i].iloc[:,-1]
        center_function = lambda x: x - x.mean()
        X = center_function(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        pca = PCA(0.70)
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        n_pcs= pca.components_.shape[0]
        pca_comp = [abs(pca.components_[x]) for x in range(n_pcs)]
        pca_comp = [list(pca_comp[l]) for l in range(n_pcs)]
        pca_comp_sort = [sorted(pca_comp[p])[-3:] for p in range(n_pcs)]

        most_important_index = []
        for k in range(len(pca_comp_sort)):
          for j in range(len(pca_comp_sort[k])):
            most_important_index.append(pca_comp[k].index(pca_comp_sort[k][j]))
        
        initial_feature_names = dfs[i].columns
        most_important_index = list(set(most_important_index))
        most_important_names = [initial_feature_names[most_important_index[q]] for q in range(len(most_important_index))]
        
        sub_col = list(most_important_names)
        sub_col.append("CONDITION")
        df_5 = dfs[i][sub_col]    
        print(dfs_names[i])
        print([dfs[i].columns.get_loc(sub_col[j]) for j in range(len(sub_col))])

        X_sub = df_5.iloc[:,:-1]
        y_sub = df_5.iloc[:,-1]
        X_sub_train, X_sub_test, y_train, y_test = train_test_split(X_sub, y_sub, test_size=0.3, random_state=42)

        # LOGISTIC REGRESSION
        lr = LogisticRegression(C=0.01, random_state=42)
        lr.fit(X_sub_train, y_train)
        y_lr_pred = lr.predict(X_sub_test)
        acc_lr = accuracy_score(y_test, y_lr_pred)
        #feture importance LR 
        importances = pd.DataFrame(data={
            'Attribute': X_sub_train.columns,
            'Importance': lr.coef_[0]
        })
        importances = importances.sort_values(by='Importance', ascending=False)
        plt.bar(x=importances['Attribute'], height=importances['Importance'], color='#087E8B')
        plt.title('Feature importances obtained from coefficients- LOGISTIC REGRESSION', size=20)
        plt.xticks(rotation='vertical')
        plt.show()

        # RANDOM FOREST
        rf = RandomForestClassifier(random_state=42)
        rf.fit(X_sub_train, y_train)
        y_rf_pred = rf.predict(X_sub_test)
        acc_rf = accuracy_score(y_test, y_rf_pred)
        #feture importance RF
        importance = rf.feature_importances_

        print(pd.DataFrame(np.c_[X_sub_train.columns, rf.feature_importances_],
                          columns = ["Feature", "Importance"]).sort_values("Importance", ascending = False))
        plt.bar(X_sub_train.columns, importance)
        plt.xticks(rotation='vertical')
        plt.show()

        #SVM
        svm = LinearSVC(loss='hinge',random_state=42)
        svm.fit(X_sub_train, y_train)
        y_pred_svm = svm.predict(X_sub_test)
        acc_svm = accuracy_score(y_pred_svm,y_test)
        importances = pd.DataFrame(data={
            'Attribute': X_sub_train.columns,
            'Importance': svm.coef_[0]
        })
        #feture importance SVM
        importances = importances.sort_values(by='Importance', ascending=False)
        plt.bar(x=importances['Attribute'], height=importances['Importance'], color='#087E8B')
        plt.title('Feature importances obtained from coefficients- SVM', size=20)
        plt.xticks(rotation='vertical')
        plt.show()

        data[dfs_names[i]] = [acc_lr,acc_rf, acc_svm]

        
    return data


In [ ]:
data = {}
results_acc = pca_ds_acc(dfs,dfs_names,data)
print(results_acc)

In [ ]:
acc_df = pd.DataFrame(results_acc)
acc_df

In [ ]:
mean_acc_df = acc_df.mean()
mean_acc_df

In [ ]:
X_axis = np.arange(len(dfs_names))
X_axis

In [ ]:
std_acc = acc_df.std()
std_acc

In [ ]:
f = plt.figure()
f.set_figwidth(24)
f.set_figheight(10)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.bar(X_axis, mean_acc_df,0.4, yerr = std_acc)
plt.xticks(X_axis, dfs_names, rotation=90)
plt.show()